## Import Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Data Processing

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#print the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding= 'ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns and reading the dataset again

column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding= 'ISO-8859-1', names=column_names)

In [ ]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
# Check for missing values

twitter_data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
# Check the distribution of target columns

twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
# Convert the target "4" to "1"

twitter_data.loc[twitter_data['target'] == 4, 'target'] = 1

In [ ]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

### Lemmatization

In [ ]:
from nltk.corpus import wordnet

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
lemmatizer= WordNetLemmatizer()

In [ ]:
lemmatizer= WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts."""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_content(content):
    lemmatized_content = re.sub('[^a-zA-Z]', ' ', content)
    lemmatized_content = lemmatized_content.lower()
    lemmatized_content = lemmatized_content.split()
    lemmatized_content = [
        lemmatizer.lemmatize(word, get_wordnet_pos(word))
        for word in lemmatized_content
        if word not in stopwords.words('english')
    ]
    lemmatized_content = ' '.join(lemmatized_content)
    return lemmatized_content

In [ ]:
twitter_data['lemmatized_text'] = twitter_data['text'].apply(lemmatize_content)

In [ ]:
twitter_data.to_csv('lemmatized_twitter_data.csv', index=False)

In [ ]:
twitter_data.head()

,target,ids,date,flag,user,text,lemmatized_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive many time ball manage save rest ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving mad see


### Load lemmatized data

In [ ]:
twitter_data= pd.read_csv('/content/drive/MyDrive/Twitter sentiment analysis/deflaut_lemmatized_twitter_data.csv')

In [ ]:
twitter_data.head()

,target,ids,date,flag,user,text,lemmatized_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive many time ball manage save rest ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving mad see


In [ ]:
twitter_data.shape

(1600000, 7)

In [ ]:
X = twitter_data['lemmatized_text']
Y = twitter_data['target']

In [ ]:
Y.value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [ ]:
print(twitter_data.dtypes)

target              int64
ids                 int64
date               object
flag               object
user               object
text               object
lemmatized_text    object
dtype: object


### Vectorize

In [ ]:
sent_length = [len(sentence.split()) for sentence in X if len(sentence.split())>30]

AttributeError: 'float' object has no attribute 'split'

Looks like, there are some float values in lemmatized_text.

In [ ]:
# Step 1: Convert floats or non-string values to string
twitter_data['lemmatized_text'] = twitter_data['lemmatized_text'].apply(lambda x: str(x) if isinstance(x, float) else x)

In [ ]:
X = twitter_data['lemmatized_text']
Y = twitter_data['target']

In [ ]:
sent_length = [len(sentence.split()) for sentence in X if len(sentence.split())>20]

In [ ]:
len(sent_length)

2197

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
MAX_FEATURES = 75000

In [ ]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                              output_sequence_length=20,
                              output_mode='int')

In [ ]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer.get_vocabulary()

In [ ]:
len(vectorizer.get_vocabulary())

75000

In [ ]:
vectorizer('Hello world, life is great')

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([349, 204,  95,   1,  46,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])>

In [ ]:
vectorizer('Hey buddy, life is awesome')

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([ 86, 909,  95,   1,  90,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])>

In [ ]:
vectorized_text = vectorizer(X.values)

In [ ]:
vectorized_text.dtype

tf.int64

In [ ]:
vectorized_text.shape

TensorShape([1600000, 20])

## Prepare Dataset

In [ ]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, Y))
dataset = dataset.cache()
dataset = dataset.shuffle(1000000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(4)

In [ ]:
train = dataset.take(int(len(dataset)*.8))
val = dataset.skip(int(len(dataset)*.8)).take(int(len(dataset)*.1))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

## Create Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
model = Sequential()
# Create the embedding layer
model.add(Embedding(MAX_FEATURES + 1, 32))
# Bidirectional LSTM Layer with specified parameters
model.add(Bidirectional(LSTM(50, activation='tanh', dropout=0.2, recurrent_dropout=0.2)))
# Dense layer with Dropout
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
# Output layer
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.build(input_shape=(None, None))

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import load_model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the filepath pattern for saving the model weights
filepath = "/kaggle/working/Twitter_epoch_{epoch:02d}_val_loss:{val_loss:.2f}_val_acc:{val_accuracy:.2f}.weights.h5"

# Create a list of callbacks
callbacks = [ModelCheckpoint(filepath=filepath,save_weights_only=True,save_freq="epoch")]

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),loss=BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 32)       │     2,400,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100)            │        33,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,446,289 (9.33 MB)

 Trainable params: 2,446,289 (9.33 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train, epochs=5, validation_data=val, callbacks=callbacks)

Epoch 1/5
72347/80000 ━━━━━━━━━━━━━━━━━━━━ 5:30 43ms/step - accuracy: 0.7937 - loss: 0.4444

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



80000/80000 ━━━━━━━━━━━━━━━━━━━━ 3552s 44ms/step - accuracy: 0.8271 - loss: 0.3845 - val_accuracy: 0.8192 - val_loss: 0.3976
Epoch 4/5
80000/80000 ━━━━━━━━━━━━━━━━━━━━ 3547s 44ms/step - accuracy: 0.8321 - loss: 0.3761 - val_accuracy: 0.8243 - val_loss: 0.3898
Epoch 5/5
 2284/80000 ━━━━━━━━━━━━━━━━━━━━ 56:07 43ms/step - accuracy: 0.8639 - loss: 0.3181

In [ ]:
Y.shape

(1600000,)

## Make Prediction

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('/content/drive/MyDrive/twitter_sentiment_analysis/twitter_sentiment_analysis_epoch4.h5')

In [ ]:
input_text = vectorizer("6 years later and nothing changes… And he blames the other driver each time.")

In [ ]:
input_text = tf.expand_dims(input_text, axis=0)

In [ ]:
input_text.shape

TensorShape([1, 20])

In [ ]:
res = model.predict(input_text)

1/1 [==============================] - 1s 801ms/step


In [ ]:
res.shape

(1, 1)

In [ ]:
res

array([[0.60668284]], dtype=float32)

In [ ]:
if res<0.5:
  print('Negative')
else:
  print('Positive')

Positive


## Save the vectorizer

In [ ]:

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorizer)

In [ ]:
model.save('/content/drive/MyDrive/twitter_sentiment_analysis/vectorizer_model', save_format='tf')

In [ ]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/twitter_sentiment_analysis/vectorizer_model')

In [ ]:
loaded_vectorizer = loaded_model.layers[0]

In [ ]:
vectorizer('Hello world, life is pretty awesome')

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([349, 204,  95,   1, 151,  90,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])>

In [ ]:
loaded_vectorizer('Hello world, life is pretty awesome')

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([349, 204,  95,   1, 151,  90,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])>